In [ ]:
import numpy as np
import os
from matplotlib import pyplot as plt
from matplotlib import gridspec

### Load the dataset

This dataset contains "pvtu_step", "step", "time", "trench (rad)", "slab depth(m)", "100km dip (rad)" as entries

In [ ]:
Ro = 6371e3  # radius of Earth
data_file = "slab_morph.txt"
assert(os.path.isfile(data_file))
data = np.loadtxt(data_file)
ts = data[:, 2]
trenches = data[:, 3]
# print(ts) # debug

### Plot the position of the trench

In [ ]:
fig = plt.figure(tight_layout=True)
gs = gridspec.GridSpec(2, 1)
ax = fig.add_subplot(gs[0, 0])
ax.plot(ts, trenches)
ax.set_xlabel('Time (yr)')
ax.set_ylabel('Trench (rad)')
# derive the velocity and plot
trench_velocities = np.gradient(trenches, ts) * Ro
ax1 = fig.add_subplot(gs[1, 0])
ax1.plot(ts, trench_velocities)
ax1.set_xlabel('Time (yr)')
ax1.set_ylabel('Trench motion (m/yr)')
fig.show()


### Resample to a uniform grid

In the following example, I resample this time series to an 1e5-yr interval.
In this way, I could perform fft transform of the series

In [ ]:
tmax = np.max(ts)
t_inter = 1e5  # resample interval
ts_re = np.arange(0.0, tmax, t_inter)
trenches_re = np.interp(ts_re, ts, trenches)
fig = plt.figure(tight_layout=True)
gs = gridspec.GridSpec(2, 1)
ax = fig.add_subplot(gs[0, 0])
ax.plot(ts_re, trenches_re)
ax.set_xlabel('Time (yr)')
ax.set_ylabel('Trench (rad)')
# derive the velocity and plot
trench_velocities_re = np.gradient(trenches_re, ts_re) * Ro
ax1 = fig.add_subplot(gs[1, 0])
ax1.plot(ts_re, trench_velocities_re)
ax1.set_xlabel('Time (yr)')
ax1.set_ylabel('Trench motion (m/yr)')
print("The number of entries in array: ", ts_re.size)
fig.show()

### Define the filter

a. FFT transform of a time series

b. Remove high-frequency entries in the spectrum

c. invert FFT to get the clean data

In [ ]:
def fft_denoiser(x, n_components, to_real=True):
    """Fast fourier transform denoiser.
    
    Denoises data using the fast fourier transform.
    
    Parameters
    ----------
    x : numpy.array
        The data to denoise.
    n_components : int
        The value above which the coefficients will be kept.
    to_real : bool, optional, default: True
        Whether to remove the complex part (True) or not (False)
        
    Returns
    -------
    clean_data : numpy.array
        The denoised data.
        
    References
    ----------
    .. [1] Steve Brunton - Denoising Data with FFT[Python]
       https://www.youtube.com/watch?v=s2K1JfNR7Sc&ab_channel=SteveBrunton
    
    """
    n = len(x)
    
    # compute the fft
    fft = np.fft.fft(x, n)
    
    # compute power spectrum density
    # squared magnitud of each fft coefficient
    PSD = fft * np.conj(fft) / n
    
    # keep high frequencies
    _mask = PSD > n_components
    fft = _mask * fft
    
    # inverse fourier transform
    clean_data = np.fft.ifft(fft)
    
    if to_real:
        clean_data = clean_data.real
    
    return clean_data

### Filter the data

In [ ]:
trenches_filtered = fft_denoiser(trenches_re, 100)
fig = plt.figure(tight_layout=True)
gs = gridspec.GridSpec(2, 1)
ax = fig.add_subplot(gs[0, 0])
ax.plot(ts_re, trenches_filtered)
ax.set_xlabel('Time (yr)')
ax.set_ylabel('Trench (rad)')
# derive the velocity and plot
trench_velocities_filtered = np.gradient(trenches_filtered, ts_re) * Ro
ax1 = fig.add_subplot(gs[1, 0])
ax1.plot(ts_re, trench_velocities_filtered)
ax1.set_xlabel('Time (yr)')
ax1.set_ylabel('Trench motion (m/yr)')
print("The number of entries in array: ", ts_re.size)
fig.show()